# Assignment 2: Application of CLIP

### Basic Imports

In [1]:
import os
import time
import os.path as osp

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader,random_split

from torchvision.datasets import CIFAR10
from torchvision import datasets
from torchvision import transforms
import torchvision

from PIL import Image, ImageFilter
import matplotlib.pyplot as plt
from PIL import Image
from clip import clip

import shutil
from sklearn.model_selection import train_test_split
from torchvision.datasets import ImageFolder

from fastai.vision.all import *

import matplotlib.pyplot as plt

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Hyperparameters

In [2]:
BATCH_SIZE = 128
VISUAL_BACKBONE = ['RN50','RN101','ViT-B/32','ViT-B/16','ViT-L/14'] # RN50, RN101, RN50x4, RN50x16, ViT-B/32, ViT-B/16, ViT-L/14

### Device

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


### Dataset
---

#### Cifar10

In [4]:
transform_cifar10_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
transform_cifar10_test = transforms.Compose([
#     transforms.Resize(size=224),
#     transforms.CenterCrop(size=(224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

train_set = torchvision.datasets.CIFAR10(root='/shareddata', train=True,
                                       download=True, transform=transform_cifar10_test)
train_dataloader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=2)
test_set = torchvision.datasets.CIFAR10(root='/shareddata', train=False,
                                       download=True, transform=transform_cifar10_test)
test_dataloader = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=2)

class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
dataset_name = 'CIFAR10'

Files already downloaded and verified
Files already downloaded and verified


#### Food101

In [4]:
transform_food101_train = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),  # Adjust normalization values
])

transform_food101_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),  # Adjust normalization values
])

# Load FOOD101 dataset
train_set = torchvision.datasets.Food101(root='/shareddata', split='train',
                                                  download=True, transform=transform_food101_train)
train_dataloader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE,
                                                       shuffle=True, num_workers=2)

test_set= torchvision.datasets.Food101(root='/shareddata', split='test',
                                                download=True, transform=transform_food101_test)
test_dataloader= torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE,
                                                      shuffle=False, num_workers=2)

# Define class names for FOOD101 dataset
class_names = train_set.classes
dataset_name = 'FOOD101'

#### Covid-19

In [5]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
])

# 定义训练集和测试集路径
train_path = 'data/train'
test_path = 'data/test'

# 加载训练集
train_set = ImageFolder(root=train_path, transform=transform)
train_dataloader = DataLoader(train_set, batch_size=128, shuffle=True, num_workers=4)

# 加载测试集
test_set = ImageFolder(root=test_path, transform=transform)
test_dataloader = DataLoader(test_set, batch_size=128, shuffle=False, num_workers=4)

class_names = ['Covid-19','No_findings','Pneumonia']
dataset_name = 'COVID-19'

#### MNIST

In [16]:
transform_MNIST_test = transforms.Compose([
    transforms.Resize(size=224),
    transforms.CenterCrop(size=(224, 224)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize((0.13251467,), (0.31048027,)),
])

train_set = torchvision.datasets.MNIST(root='/shareddata', train=True,
                                     download=True, transform=transform_MNIST_test)
train_dataloader = torch.utils.data.DataLoader(train_set, batch_size=128,
                                         shuffle=False, num_workers=2)
test_set = torchvision.datasets.MNIST(root='/shareddata', train=False,
                                     download=True, transform=transform_MNIST_test)
test_dataloader = torch.utils.data.DataLoader(test_set, batch_size=128,
                                         shuffle=False, num_workers=2)
class_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
dataset_name = 'MNIST'

### Model

In [5]:
# Load the model
def clipModel(VISUAL):
    model, preprocess = clip.load(name=VISUAL, device=device, download_root='/shareddata/clip/')
    model.to(device)
    return model

### Encode prompt

In [6]:
prompt = 'A photo of a' # you can try different prompt

def prompt_encode(prompt):
    """
    Args:
        prompt (str): the text prefix before the class

    Returns:
        text_inputs(torch.Tensor)

    """
    text_inputs = torch.cat([clip.tokenize(f"{prompt} {c}") for c in class_names]).to(device) 

    return text_inputs


### Zero-shot inference

In [7]:
def model_inference(model, image, text_inputs):

    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text_inputs)
    
        logits,_=model(image,text_inputs)

    return logits

### Zero-shot accuracy calculation

In [ ]:
perf = []
for i in range(len(VISUAL_BACKBONE)):
    model = clipModel(VISUAL_BACKBONE[i])
    with torch.no_grad():
        model.eval()

        start_time = time.time()

        var_correct = 0
        for image, target in test_dataloader:
            image = image.to(device)
            target = target.to(device)

            _, preds = torch.max(model_inference(model, image, prompt_encode(prompt)), 1)

            var_correct += torch.sum(preds == target.data)
        
        end_time = time.time()
        elapsed_time = end_time - start_time

        val_acc = var_correct / len(test_set)
        perf.append({"accuracy": val_acc, "time": elapsed_time})

        print(f"the zero-shot performance on {dataset_name} is {val_acc*100:.2f}%, visual encoder is {VISUAL_BACKBONE[i]}.")
        print(f"Time taken for this loop: {elapsed_time} seconds")

the zero-shot performance on FOOD101 is 68.27%, visual encoder is RN50.
Time taken for this loop: 122.24733400344849 seconds
the zero-shot performance on FOOD101 is 72.58%, visual encoder is RN101.
Time taken for this loop: 161.1329128742218 seconds
the zero-shot performance on FOOD101 is 77.85%, visual encoder is ViT-B/32.
Time taken for this loop: 78.58697891235352 seconds
the zero-shot performance on FOOD101 is 84.17%, visual encoder is ViT-B/16.
Time taken for this loop: 186.66975164413452 seconds


## Baseline model
---

### LogisticRegression:

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score


start_time = time.time()

# X_train = train_set.data
# y_train = train_set.targets
# X_test = test_set.data
# y_test = test_set.targets

# X_train = X_train.reshape((X_train.shape[0], -1))
# X_test = X_test.reshape((X_test.shape[0], -1))

X_train, y_train = [], []
X_test, y_test = [], []

# Iterate through the training data loader to get images and labels
for images, labels in train_dataloader:
    X_train.append(images.view(images.size(0), -1).numpy())
    y_train.append(labels.numpy())

# Iterate through the test data loader to get images and labels
for images, labels in test_dataloader:
    X_test.append(images.view(images.size(0), -1).numpy())
    y_test.append(labels.numpy())

# Concatenate the batches to get the full datasets
X_train = np.concatenate(X_train, axis=0)
y_train = np.concatenate(y_train, axis=0)
X_test = np.concatenate(X_test, axis=0)
y_test = np.concatenate(y_test, axis=0)

# def dataset_to_numpy(dataset):
#     data = []
#     labels = []
#     for i in range(len(dataset)):
#         sample = dataset[i]
#         data.append(sample[0].view(1, -1).numpy())
#         labels.append(sample[1])
#     return np.stack(data), np.array(labels)

# X_train, y_train = dataset_to_numpy(train_set)
# X_test, y_test = dataset_to_numpy(test_set)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=10000)
model.fit(X_train, y_train)

training_time = time.time() - start_time

start_time = time.time()
y_pred = model.predict(X_test)

testing_time = time.time() - start_time

test_accuracy = accuracy_score(y_test, y_pred)

print("Test accuracy:", test_accuracy)
print("Training time:", training_time, "seconds")
print("Testing time:", testing_time, "seconds")

Test accuracy: 0.7244444444444444
Training time: 348.6578907966614 seconds
Testing time: 0.08835625648498535 seconds


### Decision Tree:

In [11]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

start_time = time.time()

# X_train = train_set.data
# y_train = train_set.targets
# X_test = test_set.data
# y_test = test_set.targets

# X_train = X_train.reshape((X_train.shape[0], -1))
# X_test = X_test.reshape((X_test.shape[0], -1))

X_train, y_train = [], []
X_test, y_test = [], []

# Iterate through the training data loader to get images and labels
for images, labels in train_dataloader:
    X_train.append(images.view(images.size(0), -1).numpy())
    y_train.append(labels.numpy())

# Iterate through the test data loader to get images and labels
for images, labels in test_dataloader:
    X_test.append(images.view(images.size(0), -1).numpy())
    y_test.append(labels.numpy())

# Concatenate the batches to get the full datasets
X_train = np.concatenate(X_train, axis=0)
y_train = np.concatenate(y_train, axis=0)
X_test = np.concatenate(X_test, axis=0)
y_test = np.concatenate(y_test, axis=0)

# def dataset_to_numpy(dataset):
#     data = []
#     labels = []
#     for i in range(len(dataset)):
#         sample = dataset[i]
#         data.append(sample[0].view(1, -1).numpy())
#         labels.append(sample[1])
#     return np.stack(data), np.array(labels)

# X_train, y_train = dataset_to_numpy(train_set)
# X_test, y_test = dataset_to_numpy(test_set)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = DecisionTreeClassifier()
model.fit(X_train, y_train)

training_time = time.time() - start_time

start_time = time.time()
predictions = model.predict(X_test)

testing_time = time.time() - start_time

accuracy = accuracy_score(y_test, predictions)

print(f'Accuracy: {accuracy}')
print("Training time:", training_time, "seconds")
print("Testing time:", testing_time, "seconds")


Accuracy: 0.6755555555555556
Training time: 94.6327064037323 seconds
Testing time: 0.011090755462646484 seconds


### Random Forest:

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

start_time = time.time()

# X_train = train_set.data
# y_train = train_set.targets
# X_test = test_set.data
# y_test = test_set.targets

# X_train = X_train.reshape((X_train.shape[0], -1))
# X_test = X_test.reshape((X_test.shape[0], -1))

X_train, y_train = [], []
X_test, y_test = [], []

# Iterate through the training data loader to get images and labels
for images, labels in train_dataloader:
    X_train.append(images.view(images.size(0), -1).numpy())
    y_train.append(labels.numpy())

# Iterate through the test data loader to get images and labels
for images, labels in test_dataloader:
    X_test.append(images.view(images.size(0), -1).numpy())
    y_test.append(labels.numpy())

# Concatenate the batches to get the full datasets
X_train = np.concatenate(X_train, axis=0)
y_train = np.concatenate(y_train, axis=0)
X_test = np.concatenate(X_test, axis=0)
y_test = np.concatenate(y_test, axis=0)

# def dataset_to_numpy(dataset):
#     data = []
#     labels = []
#     for i in range(len(dataset)):
#         sample = dataset[i]
#         data.append(sample[0].view(1, -1).numpy())
#         labels.append(sample[1])
#     return np.stack(data), np.array(labels)

# X_train, y_train = dataset_to_numpy(train_set)
# X_test, y_test = dataset_to_numpy(test_set)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = RandomForestClassifier(n_estimators=100)
model.fit(X_train, y_train)

training_time = time.time() - start_time

start_time = time.time()
predictions = model.predict(X_test)

testing_time = time.time() - start_time

accuracy = accuracy_score(y_test, predictions)

print(f'Accuracy: {accuracy}')
print("Training time:", training_time, "seconds")
print("Testing time:", testing_time, "seconds")


Accuracy: 0.7422222222222222
Training time: 23.756004095077515 seconds
Testing time: 0.019083261489868164 seconds


### SVM

In [18]:
import time
import numpy as np
from sklearn import svm
from sklearn.metrics import accuracy_score
from keras.datasets import mnist
(x_train_image,y_train_label), (x_test_image,y_test_label) = mnist.load_data()

x_train = x_train_image.reshape(60000, 784).astype('float32') / 255
x_test = x_test_image.reshape(10000, 784).astype('float32') / 255
clf = svm.SVC(C=5.0, kernel='rbf', gamma=0.05)

start_time = time.time()
clf.fit(x_train, y_train_label)
print("Training time: {} seconds".format(time.time() - start_time))
y_pred = clf.predict(x_test)

accuracy = accuracy_score(y_test_label, y_pred)
print("Accuracy: {}%".format(accuracy * 100))

2023-12-02 08:52:59.240099: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-02 08:52:59.335934: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-02 08:52:59.773732: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-02 08:52:59.773760: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-02 08:52:59.776492: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

11490434/11490434 [==============================] - 1s 0us/step
Training time: 526.8241138458252 seconds
Accuracy: 98.37%


### CNN:

In [7]:
from tqdm import tqdm

# random seed
SEED = 1 
NUM_CLASS = 10

# Training
BATCH_SIZE = 128
NUM_EPOCHS = 15
EVAL_INTERVAL=1
SAVE_DIR = './log'

# Optimizer
LEARNING_RATE = 1e-1
MOMENTUM = 0.9
STEP=5
GAMMA=0.5

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 4, 3)  
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(4, 8, 3)  
        self.fc1 = nn.Linear(8 * 54 * 54, 32)
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 8 * 54 * 54)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = ConvNet()
model = model.to(device)

optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=STEP, gamma=GAMMA)

criterion = nn.CrossEntropyLoss()

def train_batch(model, image, target):    
    output = model(image)
    loss = criterion(output,target)
    return output, loss
def test_batch(model, image, target):
    output = model(image)
    loss = criterion(output,target)
    return output, loss

start_train_time = time.time()

for epoch in range(NUM_EPOCHS):
    model.train()
    torch.cuda.empty_cache()

    train_data_loader = tqdm(train_dataloader, desc=f'Epoch {epoch + 1}/{NUM_EPOCHS}', dynamic_ncols=True)

    for batch_idx, (image, target) in enumerate(train_data_loader):
        image = image.to(device)
        target = target.to(device)

        outputs, loss = train_batch(model, image, target)
        _, preds = torch.max(outputs, 1)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    scheduler.step()

    train_data_loader.close()

end_train_time = time.time()

train_time = end_train_time - start_train_time
print(f"Training time: {train_time:.2f} seconds")

model.eval()

val_loss = 0.0
val_corrects = 0

start_test_time = time.time()

test_data_loader = tqdm(test_dataloader, desc='Testing', dynamic_ncols=True)

for batch_idx, (image, target) in enumerate(test_data_loader):
    image = image.to(device)
    target = target.to(device)

    outputs, loss = test_batch(model, image, target)
    _, preds = torch.max(outputs, 1)

    val_corrects += torch.sum(preds == target.data)

test_data_loader.close()

end_test_time = time.time()

test_time = end_test_time - start_test_time
print(f'Testing time: {test_time:.2f} seconds')

val_acc = val_corrects.double() / len(test_set)
print(f'Acc: {val_acc:.4f}')

Epoch 15/15: 100%|██████████| 592/592 [02:11<00:00,  4.49it/s]


Training time: 1990.22 seconds


Testing: 100%|██████████| 198/198 [00:52<00:00,  3.79it/s]

Testing time: 52.31 seconds
Acc: 0.0099


In [9]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
from pathlib import Path
import numpy as np
from tqdm import tqdm

# 设置路径和数据增强
path = Path('data')
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 加载训练集
train_set = datasets.ImageFolder(root=path / 'train', transform=transform)
train_dataloader = DataLoader(train_set, batch_size=128, shuffle=True, num_workers=4)

# 加载验证集
valid_set = datasets.ImageFolder(root=path / 'test', transform=transform)
valid_dataloader = DataLoader(valid_set, batch_size=128, shuffle=False, num_workers=4)

# 定义卷积块
def conv_block(ni, nf, size=3, stride=1):
    for_pad = lambda s: s if s > 2 else 3
    return nn.Sequential(
        nn.Conv2d(ni, nf, kernel_size=size, stride=stride, padding=(for_pad(size) - 1)//2, bias=False),
        nn.BatchNorm2d(nf),
        nn.LeakyReLU(negative_slope=0.1, inplace=True)
    )

# 定义三次卷积
def triple_conv(ni, nf):
    return nn.Sequential(
        conv_block(ni, nf),
        conv_block(nf, ni, size=1),
        conv_block(ni, nf)
    )

# 定义模型
class XRayModel(nn.Module):
    def __init__(self):
        super(XRayModel, self).__init__()
        self.model = nn.Sequential(
            conv_block(3, 8),
            nn.MaxPool2d(2, stride=2),
            conv_block(8, 16),
            nn.MaxPool2d(2, stride=2),
            triple_conv(16, 32),
            nn.MaxPool2d(2, stride=2),
            triple_conv(32, 64),
            nn.MaxPool2d(2, stride=2),
            triple_conv(64, 128),
            nn.MaxPool2d(2, stride=2),
            triple_conv(128, 256),
            conv_block(256, 128, size=1),
            conv_block(128, 256),
            nn.Conv2d(256, 3, kernel_size=3, stride=1, padding=1),
            nn.Flatten(),
            nn.Linear(507, 3)
        )

    def forward(self, x):
        return self.model(x)

# 创建模型和优化器
model = XRayModel()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# 训练模型
epochs = 25
for epoch in range(epochs):
    model.train()
    total_loss = 0.0
    corrects = 0

    for images, labels in tqdm(train_dataloader, desc=f'Epoch {epoch + 1}/{epochs}'):
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        corrects += torch.sum(preds == labels.data)

    epoch_loss = total_loss / len(train_dataloader)
    accuracy = corrects.double() / len(train_set)
    print(f'Train Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.4f}')

# 验证模型
model.eval()
val_loss = 0.0
val_corrects = 0

with torch.no_grad():
    for images, labels in tqdm(valid_dataloader, desc='Validation'):
        outputs = model(images)
        loss = criterion(outputs, labels)
        val_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        val_corrects += torch.sum(preds == labels.data)

val_loss /= len(valid_dataloader)
val_accuracy = val_corrects.double() / len(valid_set)
print(f'Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}')


Epoch 1/25: 100%|██████████| 8/8 [00:08<00:00,  1.10s/it]


Train Loss: 1.1781, Accuracy: 0.4667


Epoch 2/25: 100%|██████████| 8/8 [00:08<00:00,  1.11s/it]


Train Loss: 0.8737, Accuracy: 0.5300


Epoch 3/25: 100%|██████████| 8/8 [00:09<00:00,  1.15s/it]


...

Train Loss: 0.3538, Accuracy: 0.8356


Epoch 25/25: 100%|██████████| 8/8 [00:17<00:00,  2.13s/it]


Train Loss: 0.3560, Accuracy: 0.8289


Validation: 100%|██████████| 2/2 [00:03<00:00,  1.86s/it]

Validation Loss: 0.4896, Accuracy: 0.8178
